In [1]:
import numpy as np
from utils import plot_time_series, plot_coefficients, plot_residuals
from utils_Fit import Fit4S, Fit4Y
from Estimate import Iter_model


data = np.load('eco_data.npz')

GDP_growth_tensor = data['GDP_growth'][:, :, np.newaxis]
UR_tensor = data['UR'][:, :, :, np.newaxis]
population_growth_tensor = data["population_growth"][:, :, :, np.newaxis]
Action_tensor = data['Action'][:, :, :, np.newaxis]
W_mat = data['W']


NN, TT, PP, _ = UR_tensor.shape
Gnum = 3
J0 = 3

Gvec = np.array([1 for i in range(NN)])
Timevec = [i for i in range(TT)]
Timevec = np.array(Timevec) / TT
Abar = np.einsum("ij, jklm->iklm", W_mat, Action_tensor)



splinetype = "b_spline"
IV_type = "1"

plot_time_series(GDP_growth_tensor[:, :, :,np.newaxis], Gvec=Gvec, mode="Y")
plot_time_series(UR_tensor, Gvec=Gvec, mode="S")

In [2]:
Stensor = UR_tensor
Ztensor = population_growth_tensor

S0 = Stensor[:, 0:1, :, :]

res_dict4S, res_dict4Y = Iter_model(
    Timevec=Timevec,
    S0mat=S0, Stensor=Stensor,
    Wmat=W_mat, Ztensor= Ztensor,
    Ytensor=GDP_growth_tensor,
    Atensor=Action_tensor, Abar= Abar,
    Gnum=3, J0=J0,
    Gvec=Gvec,
    Gvec_real=None,
    splinetype=splinetype,
    IV_type=IV_type,
    max_iter=100, tol=1e-6, oracle=False
)

plot_coefficients(res_dict4S, dgp=None, mode="S", plot_true=False)
plot_coefficients(res_dict4Y, dgp=None, mode="Y", plot_true=False)

No Gvec real is given.
Converged after 2 iterations.


In [3]:
StensorFit = Fit4S(res_dict4S, S0mat=S0, Stensor=Stensor,
                   Ztensor=Ztensor, Atensor=Action_tensor, Wmat=W_mat)
YtensorFit = Fit4Y(res_dict4Y, Stensor=Stensor, Atensor=Action_tensor, Wmat=W_mat)

plot_residuals(Tensor=Stensor, Tensor_fit=StensorFit, mode="S")
plot_residuals(Tensor=GDP_growth_tensor, Tensor_fit=YtensorFit, mode="Y")

In [4]:
res_dict4S["Gvec"]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)